In [ ]:
# Import libraries 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Load data
data = pd.read_csv("PUMFCCHS201718.csv")
data2 = pd.read_csv('PUMFCCHS201516.csv')

In [ ]:
# Data wrangling 

# Select columns of interest
columns_filter = ['DHH_SEX', 'DOCMH', 'CMH_005', 'CMHG010', 'CMH_015A', 'CMH_015B', 'CMH_015C', 'CMH_015D', 'CMH_015E', 'CMH_015F', 'SDC_015']

# Filter data 
filter_data1 = data.loc[:, columns_filter]
filter_data2 = data2.loc[:,columns_filter]

#Filter population
filter_data1 = filter_data1.loc[filter_data1['DHH_SEX'] == 2] #females
filter_data1 = filter_data1.loc[filter_data1['CMH_005'] == 1] #consulted a professional in past 12 months 

filter_data2 = filter_data2.loc[filter_data2['DHH_SEX'] == 2] #females
filter_data2 = filter_data2.loc[filter_data2['CMH_005'] == 1] #consulted a professional in past 12 months 

#Rename columns
new_names = ['sex','consulted_a_prof', 'past_12','num_times','Family Dr','Psychiatrist','Psychologist','Nurse','Social Worker','Other', 'aboriginal_identity']
filter_data1.columns = new_names
filter_data2.columns = new_names


#Rename values in columns
columns_to_use = ['consulted_a_prof', 'past_12','Family Dr','Psychiatrist','Psychologist','Nurse','Social Worker','Other']
filter_data1[columns_to_use] = filter_data1[columns_to_use].replace({1:1, 2: 0, 6:0, 7:0, 8:0, 9:0})
filter_data1['num_times'] = filter_data1['num_times'].replace(96, 0)

filter_data2[columns_to_use] = filter_data2[columns_to_use].replace({1:1, 2: 0, 6:0, 7:0, 8:0, 9:0})
filter_data2['num_times'] = filter_data2['num_times'].replace(96, 0)

aboriginal_women_1 = filter_data1.loc[filter_data1['aboriginal_identity'] == 1]
non_aboriginal_women_1 =filter_data1.loc[filter_data1['aboriginal_identity'] == 2]

aboriginal_women_2 = filter_data2.loc[filter_data2['aboriginal_identity'] == 1]
non_aboriginal_women_2 =filter_data2.loc[filter_data2['aboriginal_identity'] == 2]

# getting the %

columns_of_interest = ['Family Dr','Psychiatrist','Psychologist','Nurse','Social Worker','Other']

def percent_data (df):
    percentage = df[columns_of_interest].mean()*100
    result = pd.DataFrame(percentage).reset_index()
    return result

abo1_df = percent_data(aboriginal_women_1)
abo2_df = percent_data(aboriginal_women_2)
nabo1_df = percent_data(non_aboriginal_women_1)
nabo2_df = percent_data(non_aboriginal_women_2)

# Rename the columns
new_column_names = ['Type of specialist', 'percentage of attendance']

abo1_df.columns = new_column_names
abo2_df.columns = new_column_names
nabo1_df.columns = new_column_names
nabo2_df.columns = new_column_names

#Add year column
abo1_df['Year'] = '2017-2018'
abo2_df['Year'] = '2015-2016'
nabo1_df['Year'] = '2017-2018'
nabo2_df['Year'] = '2015-2016'

In [ ]:
#Joining dataframes
percent_aboriginal_females = pd.concat([abo1_df,abo2_df], axis=0)
percent_nonaboriginal_females = pd.concat([nabo1_df,nabo2_df], axis=0)

In [ ]:
#Sort values
percent_aboriginal_females = percent_aboriginal_females.sort_values(by='percentage of attendance')
percent_nonaboriginal_females = percent_nonaboriginal_females.sort_values(by='percentage of attendance')

In [ ]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [ ]:
# List of specialist
specialists = percent_aboriginal_females['Type of specialist'].unique().tolist()

#Initiate a dictionary to plot
data = {"line_x": [], "line_y": [], "2015-2016": [], "2017-2018": []}

# Iterate through the specialists
for spe in specialists:
    # Retrieve the datra for each year based on the specialist
    data["2015-2016"].extend([percent_aboriginal_females.loc[(percent_aboriginal_females['Year']=='2015-2016') & (percent_aboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0]])
    data["2017-2018"].extend([percent_aboriginal_females.loc[(percent_aboriginal_females['Year']=='2017-2018') & (percent_aboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0]])
    
    # Update with % of attendance for both years, add None to create dsiconnected line segments
    data['line_x'].extend(
        [
            percent_aboriginal_females.loc[(percent_aboriginal_females['Year']=='2015-2016') & (percent_aboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0],
            percent_aboriginal_females.loc[(percent_aboriginal_females['Year']=='2017-2018') & (percent_aboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0],
            None,
        ]

    )
    data['line_y'].extend([spe,spe,None]),

# Make plot
fig = go.Figure(
    data=[
        go.Scatter(
            x=data["line_x"],
            y=data["line_y"],
            mode="lines",
            showlegend=False,
            marker=dict(
                color="grey"
            )
        ),
        go.Scatter(
            x=data["2015-2016"],
            y=specialists,
            mode="markers",
            name="2015-2016",
            marker=dict(
                color='rgba(0,0,0,0)',
                size=10,
                line=dict(color='#FF8C00', width=2)

            )
            
        ),
        go.Scatter(
            x=data["2017-2018"],
            y=specialists,
            mode="markers",
            name="2017-2018",
            marker=dict(
                color="#FF8C00",
                size=15
            )   
        ),
    ]
)

# Customize graph
fig.update_layout(width=800, height=600) 
fig.update_xaxes(title_text='Percentage of Females', range=[0, 70], titlefont=dict(size=20))
fig.update_yaxes(tickfont=dict(size=15))
fig.update_layout(
    legend=dict(
        font=dict(
            size=15  
    )
))
fig.update_layout(
    title=dict(
        text='Figure 3. Aboriginal Females: Specialist Consultations by Year',     
        font=dict(         
            size=23,
            color='#000000'
        ),  
))

fig.show()
fig.write_image("images/fig1.jpeg")

In [ ]:
specialists = percent_nonaboriginal_females['Type of specialist'].unique().tolist()

data = {"line_x": [], "line_y": [], "2015-2016": [], "2017-2018": []}

for spe in specialists:
    data["2015-2016"].extend([percent_nonaboriginal_females.loc[(percent_nonaboriginal_females['Year']=='2015-2016') & (percent_nonaboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0]])
    data["2017-2018"].extend([percent_nonaboriginal_females.loc[(percent_nonaboriginal_females['Year']=='2017-2018') & (percent_nonaboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0]])
    data['line_x'].extend(
        [
            percent_nonaboriginal_females.loc[(percent_nonaboriginal_females['Year']=='2015-2016') & (percent_nonaboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0],
            percent_nonaboriginal_females.loc[(percent_nonaboriginal_females['Year']=='2017-2018') & (percent_nonaboriginal_females['Type of specialist'] == spe)]['percentage of attendance'].values[0],
            None,
        ]

    )
    data['line_y'].extend([spe,spe,None]),

fig = go.Figure(
    data=[
        go.Scatter(
            x=data["line_x"],
            y=data["line_y"],
            mode="lines",
            showlegend=False,
            marker=dict(
                color="grey"
            )
        ),
        go.Scatter(
            x=data["2015-2016"],
            y=specialists,
            mode="markers",
            name="2015-2016",
            marker=dict(
                color='rgba(0,0,0,0)',
                size=10,
                line=dict(color='#3BB9FF', width=2)

            )
            
        ),
        go.Scatter(
            x=data["2017-2018"],
            y=specialists,
            mode="markers",
            name="2017-2018",
            marker=dict(
                color="#3BB9FF",
                size=15
            )   
        ),
    ]
)

fig.update_layout(width=800, height=600) 
fig.update_xaxes(title_text='Percentage of Females', range=[0, 70], titlefont=dict(size=20))
fig.update_yaxes(tickfont=dict(size=15))
fig.update_layout(
    legend=dict(
        font=dict(
            size=15  
    )
))
fig.update_layout(
    title=dict(
        text='Figure 4. Non Aboriginal Females: Specialist Consultations by Year',     
        font=dict(         
            size=23,
            color='#000000'
        ),  
))

fig.show()
fig.write_image("images/fig2.jpeg")